# **2 - Temaplate Training**
## Template training using Mask R-CNN
*By Author's Name*

In this notebook we train the surface model

In [ ]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import json
import os
import sys

with open("settings.json") as s:
    settings = json.load(s)
    
ROOT_DIR = settings["META"]["ROOT_DIR"]
sys.path.append(ROOT_DIR)

# Configurations

In [ ]:
from roadlearning.configurations import json_loader
config = json_loader(settings["CONFIGS"]["BASE"]) # change "" to config name.

# Dataset

In [ ]:
from roadlearning.dataset import Dataset

In [ ]:
dataset = Dataset(source=config.NAME)
dataset.load_data(**settings["DATASET_LOADS"]["BASE"]) # Change Here

In [ ]:
print(dataset)

**Split into Train and Validation**

In [ ]:
dataset.split([0.6, 0.2], seed=settings["META"]["SEED"])

**Training and Validation splits**

In [ ]:
train_dataset = dataset.get_train()

print("Training Dataset")
print(train_dataset)

In [ ]:
valid_dataset = dataset.get_validation()

print("Validation Dataset")
print(valid_dataset)

**Remove the original dataset as we are not using it anymore**

In [ ]:
del dataset

# Mask R-CNN Model

In [ ]:
import mrcnn.model as modellib

#### Initialize Model

In [ ]:
model = modellib.MaskRCNN(mode="training", config=config, model_dir="./logs")

In [ ]:
print(config)

#### Load weights

Either load from previous models, another model, or ignore to start with random weights

In [ ]:
from mrcnn import utils

# Download COCO trained weights from Releases if needed
COCO_MODEL_PATH = os.path.join("../../../../COCO", "mask_rcnn_coco.h5")
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

# Load last model 
# model.load_weights(model.find_last(), by_name=True)

# Load spesific model
# model_path = ""
# model.load_weights(model_path, by_name=True)

#### Train Model

In [ ]:
model.train(train_dataset, valid_dataset, learning_rate=config.LEARNING_RATE, epochs=10, layers='all')